# NER

In [1]:
pip install transformers==2.6.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

data = pd.read_csv("../dataset/ner_dataset.csv", encoding="latin1").fillna(method="ffill")

In [3]:
tag_values = list(set(data["Tag"].values))
tag_values.append("PAD")
tag_values.sort()
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [4]:
import torch
from transformers import BertTokenizer, BertConfig


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'GeForce GTX 1050'

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
import transformers
from transformers import BertForTokenClassification, AdamW

In [7]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)
model.cuda();


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [9]:
output_dir = '../output/ner'
model = BertForTokenClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

# Train Sentence

In [10]:
test_sentence = "Trains Available from Delhi to Mumbai"

In [11]:
model.eval()
input_ids = []
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence]).cuda()
print(input_ids)

tensor([[  101, 20223, 11651,  8009,  2165,  1121,  6175,  1106,  8601,   102]],
       device='cuda:0')


In [12]:
with torch.no_grad():
    output = model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

In [13]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)


In [14]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))


O	[CLS]
O	Trains
O	Available
O	from
B-geo	Delhi
O	to
B-geo	Mumbai
O	[SEP]


#Intent

In [22]:
import torch
 
if torch.cuda.is_available():    
     
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
 
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



import pandas as pd
import numpy as np
train=pd.read_csv("../dataset/train.csv")
labels = train.intent.values
input_labels = []
k = 1;
mp = {}
mpp= {}
for sent in labels:
    mp[sent]=0;
for sent in labels:
    if(mp[sent] == 0):
        mp[sent] = k
        mpp[k]=sent
        k = k + 1
    input_labels.append(mp[sent]-1)

from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, random_split

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",  
    num_labels = k-1,    
    output_attentions = False,  
    output_hidden_states = False,
)

model.cuda()

output_dir = '../output/intent/'
model1 = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer1 = BertTokenizer.from_pretrained(output_dir)
model1.to(device)
tokenizer1 = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model1.to(device)
sentences = [
  test_sentence,
]
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer1.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 64,           
                        pad_to_max_length = True,
                        return_attention_mask = True,    
                        return_tensors = 'pt', 
                        truncation=True,     
                )    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

batch_size = 32  

prediction_data = TensorDataset(input_ids, attention_masks )
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
model1.eval()
predictions , true_labels = [], []
for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask  = batch
    with torch.no_grad():
        outputs = model1(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    predictions.append(logits)

print('    DONE.')

flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
intent_output=mpp[flat_predictions[0]+1]
print(intent_output)


There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1050
Predicting labels for 1 test sentences...
    DONE.
TrainAvailable


In [23]:
database=pd.read_csv("../dataset/database.csv")
database.head()
database.columns

Index(['From', 'To', 'Time', 'TrainName', 'Distance', 'Fare ', 'TrainNumber',
       'Location '],
      dtype='object')

In [24]:
print(intent_output)

TrainAvailable


In [25]:
entities={}
flag = False;
for key, word in zip(new_labels,new_tokens):
  if(key == "B-geo" and flag==False):
    entities["From"] = word	
    flag = True;
  if(key=="B-geo" and flag):
    entities["To"] = word
  if(key == "B-tim" and  word.isdigit()==False):
    entities["Time"] = word
  if(len(word)==5 and word.isdigit()):
      entities["TrainNumber"]=word
  
print(entities)

{'From': 'Delhi', 'To': 'Mumbai'}


In [26]:
def CallIntent(intent_output,entities):
  switcher = { 
        "Trainlocation": TrainLocation,
        "TrainAvailable": TrainAvailable,
        "GetDistance": GetDistance,
        "TrainRoute": TrainRoute,
        "TrainFare": TrainFare,
        "BookTickets": BookTickets
    } 
  item=switcher.get(intent_output)
  if(callable(item)):
    item(entities)
  # return switcher.get(intent_output,"nothing") 
  # return switcher[intent_output]


In [27]:
def GetDistance(entities):
  for i,row in database.iterrows():
    if(entities["From"].lower()==row["From"].lower() and entities["To"].lower()==row["To"].lower() and entities["Time"].lower()==row["Time"].lower()):
      print("The distance from {} to {} is {}" .format(entities["From"],entities["To"],row["Distance"]))

In [28]:
def TrainAvailable(entities):
  print("Trains Available from {} to {} is" .format(entities["From"],entities["To"]))
  print("TrainNo\tTrainName\tTime\tFare")
  for i,row in database.iterrows():
    if(entities["From"].lower()==row["From"].lower() and entities["To"].lower()==row["To"].lower()):
      print("{}\t{}\t{}\t{}" .format(row["TrainNumber"],row["TrainName"],row["Time"],row["Fare "]))

In [29]:
def TrainLocation(entities):
  for i,row in database.iterrows():
    if(int(entities["TrainNumber"])==row["TrainNumber"]):
      print("Location of this train is {}" .format(row["Location "]))

In [30]:
def TrainRoute(entities):
  if "TrainNumber" in entities.keys():
    for i,row in database.iterrows():
      if(int(entities["TrainNumber"])==row["TrainNumber"]):
        print("This train runs from {} to {}" .format(row["From"],row["To"]))
    else:
      TrainAvailable(entities)  

In [31]:
def TrainFare(entities):
  for i,row in database.iterrows():
    if(entities["From"].lower()==row["From"].lower() and entities["To"].lower()==row["To"].lower()):
      print("The fare from {} is {} is {}" .format(entities["From"],entities["To"],row["Fare "],))

In [32]:
def BookTickets(entities):
   TrainAvailable(entities) 
   trainno=input("Input the Train Number For booking tickets ")
   if(len(trainno)==5 and trainno.isdigit()==True):
      for i,row in database.iterrows():
        if(int(trainno)==row["TrainNumber"]):
          print("Ticket for {} is Booked Successfully" .format(row["TrainName"]))


In [33]:
CallIntent(intent_output,entities)

Trains Available from Delhi to Mumbai is
TrainNo	TrainName	Time	Fare
12345	Delhi express	Morning	1298
23456	Rajdhani	Evening	1379
